In [1]:
import pandas as pd
import requests
import zipfile
from io import BytesIO
from datetime import datetime

In [2]:
############################################################################################
def conexao(url_do_zip, max_tentativas = 50):
          
    for tentativa in range(max_tentativas):
        try:
            response = requests.get(url_do_zip, stream=True, timeout=1)
            response.raise_for_status()  # Verifica se houve algum erro no download
            print("Conexão aceita na {} tentativa".format(tentativa + 1))
            return response
        except requests.exceptions.RequestException as e:
            if tentativa+1 == max_tentativas:
                print("Erro reportado: {}", e)
    else:
        print("Falha ao fazer a conexão para o download do arquivo zip após {} tentativas.".format(max_tentativas))
        return ''
############################################################################################
def zip_dos_dados(response):
    
    if response.status_code == 200:
        zip_data = BytesIO(response.content)
        return zip_data
    else:
        print("Falha ao fazer o download do arquivo zip.")
        return ''
############################################################################################
def csv_dos_dados(zip_data, nome_do_arquivo_csv):

    with zipfile.ZipFile(zip_data, 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if nome_do_arquivo_csv in filename and filename.endswith('.csv'):
                with zip_ref.open(filename) as csv_file:
                    df = pd.read_csv(csv_file, sep=';', encoding='utf-16', 
                                     dayfirst=True, engine='c',
                                     low_memory=False)
                    return df
        print("O arquivo CSV não foi encontrado no arquivo zip.")
        return ''

def carrega_dado(url_do_zip,nome_do_arquivo_csv):
    response = conexao(url_do_zip)
    zip_data = zip_dos_dados(response)
    df = csv_dos_dados(zip_data, nome_do_arquivo_csv)
    print(df.shape)
    return df

In [3]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2020.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
df = carrega_dado(url_do_zip, nome_do_arquivo_csv)
df_consolidado = df.copy()
print(df_consolidado.shape)

Conexão aceita na 1 tentativa
(111357, 21)
(111357, 21)


In [4]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2021.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
df = carrega_dado(url_do_zip, nome_do_arquivo_csv)
df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
print(df_consolidado.shape)

Conexão aceita na 3 tentativa
(79543, 21)
(190900, 21)


In [5]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2022.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
df = carrega_dado(url_do_zip, nome_do_arquivo_csv)
df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
print(df_consolidado.shape)

Conexão aceita na 2 tentativa
(69504, 21)
(260404, 21)


In [6]:
df_filtrado = df_consolidado[(df_consolidado['Esfera'] == 'Federal') 
                  & (df_consolidado['Situacao'] == 'Concluída') 
                  & (df_consolidado['OrigemSolicitacao'] == 'Internet')]
print(df_filtrado.shape)

(259212, 21)


In [7]:
df_campos_alvo = df_filtrado[['OrgaoDestinatario', 'DataRegistro', 'DetalhamentoSolicitacao']].drop_duplicates()
contagem_ocorrencias = df_campos_alvo['OrgaoDestinatario'].value_counts()
contagem_acumulada = contagem_ocorrencias.cumsum()
total_ocorrencias = contagem_ocorrencias.sum()
percentual_acumulado = (contagem_acumulada / total_ocorrencias) * 100

In [8]:
df_contagens = pd.DataFrame({   'OrgaoDestinatario': contagem_ocorrencias.index,
                                'Qtd_pedidos_2020_2023': contagem_ocorrencias.values,
                                'PercentualAcumulado_2020_2023': percentual_acumulado.values})
df_campos_alvo = df_campos_alvo.merge(df_contagens, on='OrgaoDestinatario', how='left')
df_campos_alvo['DataRegistro'] = pd.to_datetime(df_campos_alvo['DataRegistro'], format='%d/%m/%Y')
df_campos_alvo['Dia da Semana'] = df_campos_alvo['DataRegistro'].dt.day_name()
df_campos_alvo['Mês'] = df_campos_alvo['DataRegistro'].dt.month_name()
df_campos_alvo['Ano'] = df_campos_alvo['DataRegistro'].dt.year
print(df_campos_alvo.shape)

(255237, 8)


In [9]:
# Quantidade de Órgãos
df_campos_alvo.OrgaoDestinatario.unique().shape

(312,)

In [10]:
df_campos_alvo.to_pickle('df_pedidos_baixados_2020_a_2023.pkl')

In [11]:
from pandarallel import pandarallel
import spacy
nlp = spacy.load("pt_core_news_lg")
stop_words = set(spacy.lang.pt.STOP_WORDS)

In [12]:
def processa_texto(texto, nlp, stop_words):
    import re
    # Remover pontuação
    texto = re.sub(r'[^\w\s]', '', texto)
    # Converter para minúsculas
    texto = texto.lower()
    # Tokenizar o texto
    doc = nlp(texto)  
    # Lematização e remoção de stopwords
    lemmas = [token.lemma_ for token in doc if token.text not in stop_words]
    # Retornar o texto pré-processado
    return ' '.join(lemmas)   

In [13]:
pandarallel.initialize(nb_workers=20, verbose=0) # ajustar de acordo com a cpu

In [14]:
%%time
df_campos_alvo['DetalhamentoSolicitacao'] = \
df_campos_alvo['DetalhamentoSolicitacao'].parallel_apply(processa_texto, 
                                                           nlp=nlp, 
                                                           stop_words=stop_words)

CPU times: total: 53.1 s
Wall time: 6min 24s


In [15]:
df_campos_alvo.to_pickle('df_pedidos_baixados_2020_a_2023_processado.pkl')